In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import pandas as pd
import time
import os

In [ ]:
def setup_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

In [ ]:
def crawl_medicines_in_category(driver, wait, category_char, all_data):
    page_num = 1
    category_count = 0
    while True:
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.grid-cols-12 a[href*='/thuoc/']")))
            print(f"Đang xử lý trang {page_num}")

            medicine_items = driver.find_elements(By.CSS_SELECTOR, "div.grid-cols-12 a[href*='/thuoc/']")

            page_medicines = []
            for item in medicine_items:
                try:
                    name = item.text.strip()
                    link = item.get_attribute("href")
                    if name and link:
                        medicine_data = {
                            "Tên thuốc": name,
                            "Link": link,
                            "Ký tự": category_char,
                            "Trang": page_num
                        }
                        page_medicines.append(medicine_data)
                        all_data.append(medicine_data)
                except Exception as e:
                    print(f"Lỗi khi lấy thông tin thuốc: {e}")
                    continue
            category_count += len(page_medicines)
            print(f"Đã lấy {len(page_medicines)} thuốc từ trang {page_num}")

            try:
                next_button_xpaths = [
                    '//*[@id="__next"]/div[1]/div[2]/div[3]/div/div[2]/div/div[4]/ul/li[9]','//*[@id="__next"]/div[1]/div[2]/div[3]/div/div[2]/div/div[4]/ul/li[8]',
                    '//*[@id="__next"]/div[1]/div[2]/div[3]/div/div[2]/div/div[4]/ul/li[7]','//*[@id="__next"]/div[1]/div[2]/div[3]/div/div[2]/div/div[4]/ul/li[6]',
                    '//*[@id="__next"]/div[1]/div[2]/div[3]/div/div[2]/div/div[4]/ul/li[4]'           
                ]
                
                next_button = None
                for xpath in next_button_xpaths:
                    try:
                        next_button = driver.find_element(By.XPATH, xpath)
                        if next_button:
                            break
                    except:
                        continue
                    
                if next_button:
                    print(f"Chuyển sang trang {page_num + 1}")
                    driver.execute_script("arguments[0].click();", next_button)
                    page_num += 1
                    time.sleep(2)
                else:
                    print(f"Đã hết trang cho ký tự {category_char}")
                    break
            except:
                print(f"Không tìm thấy nút 'Trang sau' tại trang {page_num}")
                break

        except TimeoutException:
            print(f"Timeout ở trang {page_num}, bỏ qua...")
            break
        except Exception as e:
            print(f"Lỗi ở trang {page_num}: {e}")
            break
    return category_count


In [ ]:
def crawl_longchau_medicine_links():
    driver = setup_driver()
    wait = WebDriverWait(driver, 15)
    
    try:
        print("Đang truy cập trang chủ Long Châu...")
        driver.get("https://nhathuoclongchau.com.vn/")
        time.sleep(3)
        
        print("Đang tìm menu Thuốc...")
        thuoc_menu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='/thuoc']")))
        driver.execute_script("arguments[0].click();", thuoc_menu)
        time.sleep(3)
        
        print("Đang tìm 'Tra cứu thuốc A-Z'...")
        driver.execute_script("window.scrollBy(0, 750);")
        time.sleep(3)
        
        tracuu = driver.find_element(By.CSS_SELECTOR, "a[href='/thuoc/tra-cuu-thuoc-a-z']")
        
        if tracuu:
            driver.execute_script("arguments[0].click();", tracuu)
        else:
            print("Không tìm thấy nút 'Tra cứu'")
            driver.quit()
            return
        time.sleep(5)
    
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.no-scrollbar")))
        driver.execute_script("window.scrollBy(0, 250);")
        print("Đang lấy danh sách ký tự A-Z...")
        
        char_buttons = driver.find_elements(By.CSS_SELECTOR, "div.no-scrollbar div.cursor-pointer")
        
        all_medicine_data = []
        
        for i in range(len(char_buttons)):
            try:
                char_buttons = driver.find_elements(By.CSS_SELECTOR, "div.no-scrollbar div.cursor-pointer")
                char_button = char_buttons[i]
                char = char_button.text.strip()
                
                if not char:
                    continue
                    
                print(f"\nĐang xử lý ký tự: {char}")
                driver.execute_script("arguments[0].click();", char_button)
                time.sleep(2)
                
                medicine_count = crawl_medicines_in_category(driver, wait, char, all_medicine_data)
                print(f"Đã thu thập {medicine_count} thuốc từ ký tự {char}")
                
            except StaleElementReferenceException:
                print(f"Element bị stale cho ký tự {char}, bỏ qua...")
                continue
            except Exception as e:
                print(f"Lỗi khi xử lý ký tự {char}: {e}")
                continue
        
        return all_medicine_data
        
    finally:
        driver.quit()





In [ ]:
def save_results(medicine_data):

    if not medicine_data:
        print("Không có dữ liệu để lưu!")
        return
    
    os.makedirs("danh_muc_links", exist_ok=True)
    df = pd.DataFrame(medicine_data)
    
    csv_file = "danh_muc_links/longchau_medicines_links.csv"
    df.to_csv(csv_file, index=False, encoding='utf-8-sig')
    print(f"Đã lưu {len(df)} thuốc vào {csv_file}")
    
    txt_file = "danh_muc_links/DS_thuoc_links.txt"
    with open(txt_file, "w", encoding="utf-8") as f:
        for link in df['Link'].unique():
            f.write(f"{link}\n")
    print(f"Đã lưu {len(df['Link'].unique())} link duy nhất vào {txt_file}")
    
    print(f"\nTHỐNG KÊ CRAWL:")
    print(f"Tổng số thuốc: {len(df)}")
    print(f"Số link duy nhất: {len(df['Link'].unique())}")
    print(f"Số ký tự đã crawl: {len(df['Ký tự'].unique())}")
    
    stats_by_char = df.groupby('Ký tự').size().sort_values(ascending=False)
    print(f"\nTHỐNG KÊ THEO KÝ TỰ:")
    for char, count in stats_by_char.items():
        print(f"{char}: {count} thuốc")

In [ ]:


if __name__ == "__main__":
    print("BẮT ĐẦU CRAWL DỮ LIỆU LONG CHÂU")
    print("=" * 50)

    medicines = crawl_longchau_medicine_links()
    save_results(medicines)
    
    print("\n" + "=" * 50)
    print("HOÀN THÀNH CRAWL DỮ LIỆU!")

BẮT ĐẦU CRAWL DỮ LIỆU LONG CHÂU
Đang truy cập trang chủ Long Châu...
Đang tìm menu Thuốc...
Đang tìm 'Tra cứu thuốc A-Z'...
Đang lấy danh sách ký tự A-Z...

Đang xử lý ký tự: 5
Đang xử lý trang 1
Đã lấy 1 thuốc từ trang 1
Đã hết trang cho ký tự 5
Đã thu thập 1 thuốc từ ký tự 5

Đang xử lý ký tự: A
Đang xử lý trang 1
Đã lấy 20 thuốc từ trang 1
Chuyển sang trang 2
Đang xử lý trang 2
Đã lấy 20 thuốc từ trang 2
Chuyển sang trang 3
Đang xử lý trang 3
Đã lấy 20 thuốc từ trang 3
Chuyển sang trang 4
Đang xử lý trang 4
Đã lấy 20 thuốc từ trang 4
Chuyển sang trang 5
Đang xử lý trang 5
Đã lấy 20 thuốc từ trang 5
Chuyển sang trang 6
Đang xử lý trang 6
Đã lấy 20 thuốc từ trang 6
Chuyển sang trang 7
Đang xử lý trang 7
Đã lấy 20 thuốc từ trang 7
Chuyển sang trang 8
Đang xử lý trang 8
Đã lấy 20 thuốc từ trang 8
Chuyển sang trang 9
Đang xử lý trang 9
Đã lấy 20 thuốc từ trang 9
Chuyển sang trang 10
Đang xử lý trang 10
Đã lấy 20 thuốc từ trang 10
Chuyển sang trang 11
Đang xử lý trang 11
Đã lấy 20 thuốc t